## SetUp

### pip install

In [ ]:
!pip install polars
!pip install optuna
!pip install colorama
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 22.4 MB/s eta 0:00:00


##### Model check

In [ ]:
from catboost import CatBoostRegressor

# 모델 로드
model = CatBoostRegressor()
model.load_model("/content/drive/MyDrive/MCTS/model_checkpoints_rs2/best_model.cbm")  # .cbm 파일 경로
# 모델 파라미터 확인
params = model.get_params()
print(params)


{'verbose': 200, 'iterations': 1000, 'loss_function': 'RMSE', 'l2_leaf_reg': 4, 'devices': '0', 'depth': 10, 'task_type': 'GPU', 'random_seed': 42, 'learning_rate': 0.1}


In [ ]:
# !sudo apt update
# !sudo apt purge *nvidia* -y
# !sudo apt install nvidia-driver-530 -y
# !pip uninstall lightgbm -y
# !pip install lightgbm --install-option=--gpu
# !git clone --recursive https://github.com/Microsoft/LightGBM
# %cd LightGBM
# !mkdir build
# %cd build
# !cmake ../../LightGBM -DUSE_GPU=1
# !make -j4
# %cd ../python-package
# !python3 setup.py install --gpu

### Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Library

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from colorama import Fore, Style
import  xgboost, catboost
import pickle

from sklearn.base import clone, BaseEstimator, TransformerMixin
from sklearn.model_selection import GroupKFold, GroupShuffleSplit, cross_val_score, cross_val_predict
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, PolynomialFeatures, SplineTransformer
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor, BaggingRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GroupKFold
from sklearn.base import clone
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np
import datetime
from catboost import CatBoostRegressor
pd.options.mode.chained_assignment = "raise"

saved_models, oof_pred = {}, {}

## Data

In [ ]:
# CSV 로딩
train = pl.read_csv('/content/drive/MyDrive/MCTS/train.csv')
test = pl.read_csv('/content/drive/MyDrive/MCTS/test.csv')

# target 열 이름을 지정
target_column = 'utility_agent1'

# target_column을 따로 분리
y_train = pd.DataFrame(train[target_column].to_pandas())  # Polars DataFrame을 Pandas로 변환 후 저장

# target_column을 제외한 train과 test의 공통 열 필터링
features = [col for col in train.columns if col in test.columns and col != target_column]

# 공통 열을 가진 test 데이터셋 생성
test = test[features]
test = test.with_columns(
    pl.lit(0).alias(target_column)  # target_column을 0으로 초기화하여 추가
)

print('Shape before dropping columns:', train.shape)

# 상수열 제거
constant_columns = np.array(train.columns)[train.select(pl.all().n_unique() == 1).to_numpy().ravel()]
print(len(constant_columns), 'columns are constant. These will be dropped.')

# 불필요한 열 삭제
drop_columns = list(constant_columns) + ['Id']
train = train.drop(drop_columns)

print('Shape after dropping columns:', train.shape)

# Null values
print('There are', train.null_count().to_numpy().sum(), 'missing values.') # 결측치

# Duplicates
print('There are', len(train) - train.n_unique(), 'duplicates.') # 중복치

# Boolean columns
print('There are', train.select(pl.all().n_unique() == 2).to_numpy().sum(), 'binary columns.') # 이진값 - TF

# Pandas로 변환 후 확인
train = train.to_pandas()
train.head(3)

Shape before dropping columns: (233234, 814)
216 columns are constant. These will be dropped.
Shape after dropping columns: (233234, 597)
There are 0 missing values.
There are 0 duplicates.
There are 382 binary columns.


,GameRulesetName,agent1,agent2,Stochastic,Asymmetric,AsymmetricForces,AsymmetricPiecesType,PlayersWithDirections,Cooperation,Team,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,0,0,0,0,0,0,0,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,0,0,0,0,0,0,0,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,0,0,0,0,0,0,0,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667


### 파생변수 추가

In [ ]:
def generate_derived_features(df):
    df['area']=df['NumRows']*df['NumColumns']
    df['row_equal_col']=(df['NumColumns']==df['NumRows']).astype(np.int8)
    df['Playouts/Moves'] = df['PlayoutsPerSecond'] / (df['MovesPerSecond'] + 1e-15)
    df['EfficiencyPerPlayout'] = df['MovesPerSecond'] / (df['PlayoutsPerSecond'] + 1e-15)
    df['TurnsDurationEfficiency'] = df['DurationActions'] / (df['DurationTurnsStdDev'] + 1e-15)
    df['AdvantageBalanceRatio'] = df['AdvantageP1'] / (df['Balance'] + 1e-15)
    df['ActionTimeEfficiency'] = df['DurationActions'] / (df['MovesPerSecond'] + 1e-15)
    df['StandardizedTurnsEfficiency'] = df['DurationTurnsStdDev'] / (df['DurationActions'] + 1e-15)
    df['AdvantageTimeImpact'] = df['AdvantageP1'] / (df['DurationActions'] + 1e-15)
    df['DurationToComplexityRatio'] = df['DurationActions'] / (df['StateTreeComplexity'] + 1e-15)
    df['NormalizedGameTreeComplexity'] =  df['GameTreeComplexity'] /  (df['StateTreeComplexity'] + 1e-15)
    df['ComplexityBalanceInteraction'] =  df['Balance'] *  df['GameTreeComplexity']
    df['OverallComplexity'] =  df['StateTreeComplexity'] +  df['GameTreeComplexity']
    df['ComplexityPerPlayout'] =  df['GameTreeComplexity'] /  (df['PlayoutsPerSecond'] + 1e-15)
    df['TurnsNotTimeouts/Moves'] = df['DurationTurnsNotTimeouts'] / (df['MovesPerSecond'] + 1e-15)
    df['Timeouts/DurationActions'] = df['Timeouts'] / (df['DurationActions'] + 1e-15)
    df['OutcomeUniformity/AdvantageP1'] = df['OutcomeUniformity'] / (df['AdvantageP1'] + 1e-15)
    df['ComplexDecisionRatio'] = df['StepDecisionToEnemy'] + df['SlideDecisionToEnemy'] + df['HopDecisionMoreThanOne']
    df['AggressiveActionsRatio'] = df['StepDecisionToEnemy'] + df['HopDecisionEnemyToEnemy'] + df['HopDecisionFriendToEnemy'] + df['SlideDecisionToEnemy']

    print("Derived features successfully created.")
    return df

# 데이터프레임 생성
df = train.copy()  # train 데이터프레임의 복사본 생성

# 파생변수 생성
train = generate_derived_features(df)
temp = generate_derived_features(df) # 다른 방식 추가용

# 생성된 파생 변수 확인
train.head(3)

Derived features successfully created.
Derived features successfully created.


,GameRulesetName,agent1,agent2,Stochastic,Asymmetric,AsymmetricForces,AsymmetricPiecesType,PlayersWithDirections,Cooperation,Team,...,DurationToComplexityRatio,NormalizedGameTreeComplexity,ComplexityBalanceInteraction,OverallComplexity,ComplexityPerPlayout,TurnsNotTimeouts/Moves,Timeouts/DurationActions,OutcomeUniformity/AdvantageP1,ComplexDecisionRatio,AggressiveActionsRatio
0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,0,0,0,0,0,0,0,...,6.060208,2.807266,71.3944,110.03,0.272184,0.001242,0.0,-0.06,0,0
1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,0,0,0,0,0,0,0,...,6.060208,2.807266,71.3944,110.03,0.272184,0.001242,0.0,-0.06,0,0
2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,0,0,0,0,0,0,0,...,6.060208,2.807266,71.3944,110.03,0.272184,0.001242,0.0,-0.06,0,0


## Grouped-KFold 우선 지정(GameRuleSet활용을 위함)

In [ ]:
# Group K-Fold 설정
crossval_kf = GroupKFold()
folds = list(crossval_kf.split(train, groups=train['GameRulesetName']))

def cross_validate_model(model, features=None, label='', save_models=False):
    """
    Perform cross-validation on the given model using the GroupKFold splits.

    Parameters:
    - model: The model to train and validate.
    - features: List of feature column names to use for training.
    - label: String label to identify the model (used in outputs).
    - save_models: Boolean to decide whether to save models from each fold.

    Returns:
    None
    """
    global oof
    if features is None:
        features = train.columns

    start_time = datetime.datetime.now()
    oof = np.full_like(y, np.nan)
    model_list = []

    for fold, (idx_tr, idx_va) in enumerate(folds):
        # Split train and validation sets
        X_tr = train.loc[idx_tr, features]
        X_va = train.loc[idx_va, features]
        y_tr = y[idx_tr]
        y_va = y[idx_va]

        # Clone and fit the model
        m = clone(model)
        m.fit(X_tr, y_tr)
        y_pred = m.predict(X_va).clip(-1, 1)

        # Save model if required
        if save_models:
            model_list.append(m)

        # Store predictions
        oof[idx_va] = y_pred
        rmse = mean_squared_error(y_va, y_pred, squared=False)
        print(f"# Fold {fold}: {rmse:=.3f}")

        # Clean up memory
        del m, X_tr, X_va, y_tr, y_va

    elapsed_time = datetime.datetime.now() - start_time
    rmse = mean_squared_error(y, oof, squared=False)
    print(f"# Overall RMSE={rmse:.3f} {label}"
          f"   {int(np.round(elapsed_time.total_seconds() / 60))} min")

    # Save models and predictions if requested
    if save_models:
        saved_models[label] = dict(features=features, model_list=model_list)
        oof_pred[label] = oof

## Data(cont)

### 유저 세부 분리

In [ ]:
%%writefile preprocess.py

def preprocess(df_pandas):
    """
    Preprocess the input Pandas DataFrame to extract features, target, and groups.

    Parameters:
    - df_pandas: Pandas DataFrame containing the dataset.

    Returns:
    - df: Processed Pandas DataFrame with additional features.
    - target: Target array (if in training mode) or None.
    - groups: Group array for GroupKFold or None.
    """
    import pandas as pd
    import numpy as np

    # Extract features from `agent1` and `agent2`
    df = df_pandas.copy()
    df['p1_selection'] = df['agent1'].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[0]
    df['p1_exploration'] = df['agent1'].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[1].astype(np.float32)
    df['p1_playout'] = df['agent1'].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[2]
    df['p1_bounds'] = df['agent1'].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[3]

    df['p2_selection'] = df['agent2'].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[0]
    df['p2_exploration'] = df['agent2'].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[1].astype(np.float32)
    df['p2_playout'] = df['agent2'].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[2]
    df['p2_bounds'] = df['agent2'].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[3]

    # Check if `utility_agent1` exists to differentiate between training and test data
    if 'utility_agent1' in df_pandas.columns:  # Training data
        target = df_pandas['utility_agent1'].values
        groups = df_pandas['GameRulesetName'].values
    else:  # Test data
        target, groups = None, None

    return df, target, groups

Overwriting preprocess.py


In [ ]:
exec(open('preprocess.py', 'r').read())

train_pd, y, groups = preprocess(train)

train = train_pd

### 게임수 가중치 추가(로그 기반)

In [ ]:
# num_games_played 계산
num_games_played = train.filter(regex='^num_.*$').sum(axis=1)

# 로그 변환을 통한 가중치 계산 (게임수 15보다 적을 경우, 최소값 설정)
weight = num_games_played.apply(lambda x: np.log(x / 15 + 1))

# 새로운 열 추가
train = pd.concat(
    [train, num_games_played.rename('num_games_played'), weight.rename('weight')],
    axis=1
)

# 데이터프레임 복사로 조각화 방지
train = train.copy()

In [ ]:
# # from catboost import CatBoostRegressor
# # import pandas as pd
# # import datetime
# # from sklearn.metrics import mean_squared_error
# # import numpy as np
# # from colorama import Fore, Style

# # GPU 지원을 위한 CatBoost 파라미터 설정
# catboost_params = {
#     'learning_rate': 0.05,
#     'iterations': 500,
#     'depth': 10,
#     'l2_leaf_reg': 3,
#     'random_state': 42,
#     'boosting_type': 'Plain',  # 'Ordered' or 'Plain'
#     'task_type': 'GPU',  # GPU 사용
#     'devices': '0',  # 사용할 GPU 번호
#     'eval_metric': 'RMSE',  # 평가 지표
# }

# # CatBoost 모델 초기화
# model = CatBoostRegressor(**catboost_params)

# # 수치형 데이터와 범주형 데이터 분리
# numeric_features = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
# cat_features = train.select_dtypes(include=['object']).columns.tolist()  # 범주형 변수

# # Feature Importance 계산 함수 정의
# def compute_catboost_feature_importance(model, train, features, y, cat_features):
#     """
#     CatBoost 내장 Feature Importance 계산.
#     """
#     start_time = datetime.datetime.now()

#     # 모델 학습
#     model.fit(train[features], y, cat_features=cat_features)
#     y_pred = model.predict(train[features]).clip(-1, 1)
#     rmse = mean_squared_error(y, y_pred, squared=False)
#     print(f"{Fore.GREEN}# Overall RMSE={rmse:.3f}{Style.RESET_ALL}")

#     # Feature Importance 추출
#     importance_df = pd.DataFrame({
#         'Feature': features,
#         'Importance': model.get_feature_importance()
#     }).sort_values(by='Importance', ascending=False)

#     elapsed_time = datetime.datetime.now() - start_time
#     print(f"{Fore.GREEN}# Time Elapsed: {int(np.round(elapsed_time.total_seconds() / 60))} min{Style.RESET_ALL}")
#     return importance_df

# # Feature Importance 계산
# importance_df = compute_catboost_feature_importance(
#     model=model,
#     train=train,
#     features=numeric_features + cat_features,  # 수치형 + 범주형 변수
#     y=y,
#     cat_features=cat_features
# )

# # 상위 Feature 출력
# print("\nTop 50 Important Features:")
# display(importance_df.head(50))

# # 중요 Feature 저장
# good_features = list(importance_df.query("Importance > 0")['Feature'])

# print("\nGood Features for Further Training:")
# print(len(good_features))


In [ ]:
good_features = ['utility_agent1',
 'num_losses_agent1',
 'num_wins_agent1',
 'num_draws_agent1',
 'Region',
 'BranchingFactorMaximum',
 'Asymmetric',
 'AnimalComponent',
 'BranchingFactorChangeSign',
 'MancalaStyle',
 'NumDiagonalDirections',
 'OutcomeUniformity',
 'NumDice',
 'DurationTurnsStdDev',
 'SowWithEffect',
 'PushEffectFrequency',
 'AggressiveActionsRatio',
 'Tile',
 'DecisionMoves',
 'NoProgressEndFrequency',
 'NumDirections',
 'GreaterThan',
 'OverallComplexity',
 'weight',
 'IsEnemy',
 'p2_selection',
 'NumCells',
 'Trigger',
 'agent1',
 'num_games_played',
 'AsymmetricPiecesType',
 'GroupEnd',
 'BoardCoverageDefault',
 'MoveDistanceChangeLineBestFit',
 'SowBacktracking',
 'Checkmate',
 'p2_playout',
 'ConnectionEndFrequency',
 'Logic',
 'LeapDecisionToEnemyFrequency',
 'NoTargetPiece',
 'ForEachPiece',
 'SetValueFrequency',
 'p1_selection',
 'NoOwnPiecesEnd',
 'agent2',
 'NumPlayPhase',
 'NumStartComponentsBoard',
 'ShootDecisionFrequency',
 'MovesPerSecond',
 'Sow',
 'HopDecisionFriendToEnemyFrequency',
 'PieceNumberChangeSign',
 'NoOwnPiecesWinFrequency',
 'Meta',
 'DecisionFactorChangeNumTimes',
 'LineDraw',
 'SlideDecisionToFriendFrequency',
 'AdjacentDirection',
 'NumPhasesBoard',
 'PromotionDecisionFrequency',
 'SlideDecisionToEnemyFrequency',
 'NumVertices',
 'AsymmetricForces',
 'DecisionFactorAverage',
 'SameDirection',
 'MovesNonDecision',
 'Playouts/Moves',
 'JanggiStyle',
 'BoardSitesOccupiedChangeNumTimes',
 'Then',
 'StateTreeComplexity',
 'BoardStyle',
 'Set',
 'MoveConditions',
 'DecisionFactorMaximum',
 'p1_playout',
 'Drawishness',
 'StepDecision',
 'MoveDistanceMaxIncrease',
 'NoMovesEnd',
 'PieceNumberAverage',
 'DoLudeme',
 'NumAdjacentDirections',
 'PieceNumberMaximum',
 'StepDecisionToEmptyFrequency',
 'Threat',
 'MarkerComponent',
 'NumColumns',
 'InterveneCaptureFrequency',
 'Priority',
 'BallComponent',
 'ProposeDecisionFrequency',
 'QueenComponent',
 'PieceNumberMaxIncrease',
 'GreaterThanOrEqual',
 'NumPlayableSites',
 'FromToDecisionFrequency',
 'Track',
 'SurroundCapture',
 'LeapDecision',
 'TriangleTiling',
 'PushEffect',
 'Directions',
 'CanMove',
 'NumComponentsType',
 'ProposeEffectFrequency',
 'StepDecisionToEnemy',
 'SingleSiteMoves',
 'OutcomeUniformity/AdvantageP1',
 'MancalaTwoRows',
 'BishopComponent',
 'ConnectionWin',
 'FortyStonesWithFourGapsBoard',
 'ShootDecision',
 'ReplacementCapture',
 'EliminatePiecesDrawFrequency',
 'FairyChessComponent',
 'BranchingFactorVariance',
 'Scoring',
 'NumOrthogonalDirections',
 'AdvantageP1',
 'Draw',
 'NumPerimeterSites',
 'EliminatePiecesWin',
 'FromToDecisionEnemyFrequency',
 'FromToDecisionEmptyFrequency',
 'LineEnd',
 'Maximum',
 'Stochastic',
 'Dice',
 'Completion',
 'NoMovesWin',
 'Roll',
 'NoOwnPiecesLoss',
 'NoPieceMover',
 'MoveDistanceMaxDecrease',
 'Comparison',
 'NoProgressDrawFrequency',
 'ReachWinFrequency',
 'DecisionFactorChangeSign',
 'BoardSitesOccupiedMaxDecrease',
 'SpaceConditions',
 'AddEffect',
 'AdvantageTimeImpact',
 'SwapOption',
 'SowBacktrackingFrequency',
 'BoardSitesOccupiedMaxIncrease',
 'FlipFrequency',
 'AlquerqueTiling',
 'NumContainers',
 'DecisionFactorVariance',
 'NoProgressDraw',
 'Balance',
 'RollFrequency',
 'SpaceEnd',
 'SetCountFrequency',
 'BoardSitesOccupiedAverage',
 'StackType',
 'Symbols',
 'BoardSitesOccupiedChangeSign',
 'Vertex',
 'FillEndFrequency',
 'SowRemove',
 'NumPlayableSitesOnBoard',
 'SwapPiecesDecision',
 'RaceEnd',
 'DurationActions',
 'PieceNumberChangeAverage',
 'TurnsNotTimeouts/Moves',
 'NumInnerSites',
 'TurnsDurationEfficiency',
 'EnglishRules',
 'SetValue',
 'NormalizedGameTreeComplexity',
 'ActionTimeEfficiency',
 'DurationToComplexityRatio',
 'AddEffectFrequency',
 'BoardSitesOccupiedMaximum',
 'AlquerqueBoard',
 'MoveDistanceChangeAverage',
 'ConditionalStatement',
 'DurationTurns',
 'Tiling',
 'BranchingFactorChangeLineBestFit',
 'BoardSitesOccupiedChangeAverage',
 'DrawFrequency',
 'DurationTurnsNotTimeouts',
 'GameRulesetName',
 'ComplexityBalanceInteraction',
 'BoardCoverageFull',
 'LudRules',
 'ChessStyle',
 'PromotionDecision',
 'Piece',
 'SowCCW',
 'CaptureEnd',
 'TwoSitesMoves',
 'NumCentreSites',
 'StepDecisionFrequency',
 'SumDice',
 'HopDecisionMoreThanOneFrequency',
 'BranchingFactorChangeMaxIncrease',
 'SowProperties',
 'BranchingFactorMedian',
 'StandardizedTurnsEfficiency',
 'TrackLoop',
 'BoardSitesOccupiedMedian',
 'PieceNumberChangeLineBestFit',
 'NoSiteMoves',
 'BranchingFactorChangeMaxDecrease',
 'PassDecisionFrequency',
 'ShogiStyle',
 'DecisionFactorMedian',
 'MoveAgainFrequency',
 'DecisionFactorMaxDecrease',
 'SowFrequency',
 'PieceState',
 'NumStartComponentsPerPlayer',
 'SetNextPlayerFrequency',
 'DurationMoves',
 'PieceValue',
 'MoveDistanceChangeSign',
 'IsFriend',
 'PieceNumberMedian',
 'MoveDistanceMedian',
 'HopDecisionEnemyToEmptyFrequency',
 'PieceConditions',
 'StarBoard',
 'NumStartComponents',
 'MoveDistanceVariance',
 'CopyContext',
 'NumOuterSites',
 'StepEffect',
 'PassDecision',
 'MancalaBoard',
 'DecisionFactorChangeLineBestFit',
 'DecisionFactorChangeAverage',
 'NumStartComponentsBoardPerPlayer',
 'PiecesPlacedOnBoard',
 'NoMoves',
 'ForwardDirection',
 'BranchingFactorChangeAverage',
 'Complement',
 'SwapPiecesDecisionFrequency',
 'PatternEndFrequency',
 'ScoreDifferenceMaxIncrease',
 'AdvantageBalanceRatio',
 'MoveDistanceAverage',
 'SetSiteState',
 'IsPieceAt',
 'Algorithmics',
 'Intersection',
 'PieceNumberMaxDecrease',
 'SlideDecisionFrequency',
 'OppositeDirection',
 'CheckmateFrequency',
 'p1_bounds',
 'SurakartaStyle',
 'EliminatePiecesWinFrequency',
 'HopDecisionEnemyToEnemyFrequency',
 'NumComponentsTypePerPlayer',
 'ScoringLossFrequency',
 'OpeningContract',
 'PieceNumberVariance',
 'HopDecisionFrequency',
 'BoardSitesOccupiedChangeLineBestFit',
 'AddDecisionFrequency',
 'StepDecisionToEnemyFrequency',
 'FromToDecisionWithinBoardFrequency',
 'ReplacementCaptureFrequency',
 'NumCorners',
 'GroupEndFrequency',
 'BoardCoverageUsed',
 'TaflComponent',
 'NumConvexCorners',
 'MoveDistanceMaximum',
 'SwapPlayersEffect',
 'EfficiencyPerPlayout',
 'HopDecisionFriendToEmptyFrequency',
 'NumStartComponentsHandPerPlayer',
 'BoardSitesOccupiedVariance',
 'ScoreDifferenceMaxDecrease',
 'RemoveEffectFrequency',
 'ScoreDifferenceMaximum',
 'NumTopSites',
 'Conditions',
 'area',
 'CheckmateWinFrequency',
 'Timeouts',
 'Fill',
 'HopCaptureMoreThanOneFrequency',
 'Timeouts/DurationActions',
 'ScoreDifferenceChangeSign',
 'LeapDecisionFrequency',
 'NumOffDiagonalDirections',
 'BackgammonStyle',
 'BackwardLeftDirection',
 'Odd',
 'DirectionCaptureFrequency',
 'LoopLoss',
 'NumConcaveCorners',
 'MancalaThreeRows',
 'SpiralShape',
 'CaptureSequenceFrequency']

In [ ]:
def separate_features(train, good_features):
    """
    Separate categorical and numeric features in the train dataset.

    Parameters:
    - train: Pandas DataFrame containing the data.
    - good_features: List of numeric features to retain.

    Returns:
    - categorical_features: DataFrame with only categorical features.
    - numeric_features: DataFrame with only the good numeric features.
    """
    # Identify categorical columns
    categorical_features = train.select_dtypes(include=['object', 'category']).copy()

    # Filter numeric features from good_features
    numeric_features = train[good_features].copy()

    return categorical_features, numeric_features

# Example usage:
categorical_features, numeric_features = separate_features(train, good_features)

In [ ]:
def combine_features(categorical_features, numeric_features):
    """
    Combine categorical and numeric features into a single DataFrame.

    Parameters:
    - categorical_features: DataFrame with categorical features.
    - numeric_features: DataFrame with numeric features.

    Returns:
    - combined_df: Combined DataFrame with both categorical and numeric features.
    """
    combined_df = pd.concat([categorical_features, numeric_features], axis=1)
    return combined_df

# Example usage
train = combine_features(categorical_features, numeric_features)

In [ ]:
def drop_columns(df, columns_to_drop):
    """
    Drop specified columns from the DataFrame.

    Parameters:
    - df: Pandas DataFrame.
    - columns_to_drop: List of column names to drop.

    Returns:
    - df: DataFrame with specified columns removed.
    """
    return df.drop(columns=columns_to_drop, axis=1, errors='ignore')

# 드랍할 컬럼 목록
columns_to_drop = [
    'GameRulesetName', 'EnglishRules', 'LudRules',
    'num_wins_agent1', 'num_draws_agent1',
    'num_losses_agent1', "agent1", "agent2"]

# 컬럼 드랍 수행
train = drop_columns(train, columns_to_drop)

# 결과 확인
print("Columns after drop:")
train

Columns after drop:


,p1_selection,p1_playout,p1_bounds,p2_selection,p2_playout,p2_bounds,utility_agent1,Region,BranchingFactorMaximum,Asymmetric,...,NumOffDiagonalDirections,BackgammonStyle,BackwardLeftDirection,Odd,DirectionCaptureFrequency,LoopLoss,NumConcaveCorners,MancalaThreeRows,SpiralShape,CaptureSequenceFrequency
0,ProgressiveHistory,MAST,false,ProgressiveHistory,Random200,false,-0.466667,0,48.00,0,...,0.0,0,0,0,0.0,0,0,0,0,0.0
1,ProgressiveHistory,MAST,false,UCB1GRAVE,NST,true,-0.333333,0,48.00,0,...,0.0,0,0,0,0.0,0,0,0,0,0.0
2,ProgressiveHistory,MAST,true,UCB1,NST,false,-0.066667,0,48.00,0,...,0.0,0,0,0,0.0,0,0,0,0,0.0
3,ProgressiveHistory,MAST,true,UCB1,NST,false,-0.333333,0,48.00,0,...,0.0,0,0,0,0.0,0,0,0,0,0.0
4,ProgressiveHistory,MAST,true,UCB1GRAVE,NST,false,-0.333333,0,48.00,0,...,0.0,0,0,0,0.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,UCB1Tuned,NST,false,ProgressiveHistory,Random200,false,-0.733333,0,65.85,0,...,0.0,0,0,0,0.0,0,0,0,0,0.0
233230,UCB1Tuned,Random200,false,UCB1,MAST,false,0.266667,0,65.85,0,...,0.0,0,0,0,0.0,0,0,0,0,0.0
233231,UCB1Tuned,Random200,false,UCB1GRAVE,NST,false,0.666667,0,65.85,0,...,0.0,0,0,0,0.0,0,0,0,0,0.0
233232,UCB1Tuned,Random200,false,UCB1GRAVE,NST,true,0.666667,0,65.85,0,...,0.0,0,0,0,0.0,0,0,0,0,0.0


In [ ]:
# train에서 중복된 컬럼 제거
train = train.loc[:, ~train.columns.duplicated()]

# 결과 확인
print("중복된 컬럼이 제거된 후 데이터프레임의 컬럼:")
print(train.columns)

중복된 컬럼이 제거된 후 데이터프레임의 컬럼:
Index(['p1_selection', 'p1_playout', 'p1_bounds', 'p2_selection', 'p2_playout',
       'p2_bounds', 'utility_agent1', 'Region', 'BranchingFactorMaximum',
       'Asymmetric',
       ...
       'NumOffDiagonalDirections', 'BackgammonStyle', 'BackwardLeftDirection',
       'Odd', 'DirectionCaptureFrequency', 'LoopLoss', 'NumConcaveCorners',
       'MancalaThreeRows', 'SpiralShape', 'CaptureSequenceFrequency'],
      dtype='object', length=298)


In [ ]:
# import pandas as pd
# import catboost
# from catboost import CatBoostRegressor, Pool
# import joblib

# # 데이터에서 중복된 컬럼 제거
# train = train.loc[:, ~train.columns.duplicated()]

# # 범주형 및 수치형 변수 분리
# categorical_columns = train.select_dtypes(include=['object', 'category']).columns
# numeric_columns = train.select_dtypes(exclude=['object', 'category']).columns

# # X (특성)에서 범주형 변수만 선택해서 CatBoost에 전달
# cat_features = categorical_columns  # 범주형 변수만 지정

# # 예시 데이터 준비 (train 데이터셋에서 'utility_agent1'을 타겟 변수로 사용)
# X_train = train.drop(columns=['utility_agent1'])  # 'utility_agent1'은 실제 타겟 컬럼명으로 바꿔야 합니다.
# y_train = train['utility_agent1']  # 실제 타겟 컬럼명으로 바꿔야 합니다.

# # # CatBoostRegressor 모델 초기화
# model = CatBoostRegressor(
#     loss_function='RMSE',
#     iterations=1000,  # 최대 반복 횟수
#     depth=10,         # 트리 깊이
#     learning_rate=0.1,
#     cat_features=[train.columns.get_loc(col) for col in cat_features],  # cat_features는 컬럼 인덱스를 사용
#     verbose=200,      # 학습 중 로그를 출력
#     thread_count=-1,  # 모든 CPU 코어를 사용
#     task_type='GPU',  # GPU 사용
#     devices='0'       # 첫 번째 GPU 사용 (0번 GPU)
# )

# # CatBoostGridSearchCV를 사용하여 하이퍼파라미터 검색
# grid_search_result = model.grid_search(
#     param_grid={
#         'iterations': [500, 1000, 1500],
#         'depth': [6, 8, 10],
#         'learning_rate': [0.05, 0.1, 0.2],
#         'l2_leaf_reg': [3, 5, 7],
#     },
#     X=X_train,
#     y=y_train,
#     cv=3,  # 3폴드 교차검증
#     refit=True,  # 최적 모델로 다시 학습
#     verbose=10   # 검색 진행 상황 출력
# )

# # 최적 하이퍼파라미터 출력
# print(f"Best Parameters: {grid_search_result['params']}")

# # 최적의 모델을 저장
# best_model = grid_search_result['model']
# best_model.save_model('catboost_best_model.cbm')  # .cbm 형식으로 모델 저장

# # 모델이 저장된 파일을 확인
# print("최적 모델이 'catboost_best_model.cbm' 파일로 저장되었습니다.")


In [ ]:
# import os
# from tqdm import tqdm
# from catboost import CatBoostRegressor, Pool, cv

# # 중복된 컬럼 제거
# train = train.loc[:, ~train.columns.duplicated()]

# # 범주형 및 수치형 변수 분리
# categorical_columns = train.select_dtypes(include=['object', 'category']).columns
# numeric_columns = train.select_dtypes(exclude=['object', 'category']).columns

# # CatBoost에 사용할 범주형 컬럼의 인덱스
# cat_features = [train.columns.get_loc(col) for col in categorical_columns]

# # 데이터 준비
# X_train = train.drop(columns=['utility_agent1'])
# y_train = train['utility_agent1']

# # 하이퍼파라미터 범위
# param_grid = {
#     'iterations': [500, 1000, 1500],
#     'depth': [6, 8, 10, 12],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'l2_leaf_reg': [1, 4, 7, 10],
#     'border_count': [32, 64, 128]
# }

# # 모델 저장 경로 설정
# save_dir = '/content/drive/MyDrive/MCTS/'
# os.makedirs(save_dir, exist_ok=True)

# # tqdm을 통한 진행 상황 표시
# param_combinations = [
#     {'iterations': it, 'depth': dp, 'learning_rate': lr, 'l2_leaf_reg': l2, 'border_count': bc}
#     for it in param_grid['iterations']
#     for dp in param_grid['depth']
#     for lr in param_grid['learning_rate']
#     for l2 in param_grid['l2_leaf_reg']
#     for bc in param_grid['border_count']
# ]

# tqdm_iter = tqdm(param_combinations, desc="Grid Search Progress")
# best_score = float('inf')  # 지금까지의 최고 성능
# best_params = None         # 지금까지의 최적 파라미터
# save_counter = 0           # 저장 주기를 관리하는 카운터
# model_counter = 0          # 저장된 모델 파일 번호

# # 현재 최적 모델 및 최고 모델 저장 경로
# best_model_path = os.path.join(save_dir, "best_model_latest.cbm")
# top_model_path = os.path.join(save_dir, "top_model_latest.cbm")

# for params in tqdm_iter:
#     # 교차 검증 실행 (catboost.cv 사용)
#     params_with_loss = params.copy()
#     params_with_loss['loss_function'] = 'RMSE'  # 필수 파라미터 추가

#     results = cv(
#         Pool(X_train, y_train, cat_features=cat_features),
#         params=params_with_loss,
#         fold_count=3,  # 3-Fold 교차 검증
#         early_stopping_rounds=50,
#         verbose=False
#     )

#     # 현재 조합의 성능
#     mean_score = results['test-RMSE-mean'].min()

#     # 지금까지의 최적 모델 업데이트
#     if mean_score < best_score:
#         best_score = mean_score
#         best_params = params_with_loss

#     # 저장 주기 확인
#     save_counter += 1
#     if save_counter % 50 == 0:
#         model_counter += 1

#         # 1. 현재까지의 최고 모델 저장
#         model.set_params(**best_params)
#         model.fit(X_train, y_train)
#         checkpoint_path = os.path.join(save_dir, f"top_model_{model_counter}.cbm")
#         model.save_model(checkpoint_path)
#         print(f"Top Model Checkpoint Saved: {checkpoint_path} with RMSE: {best_score}")

#         # 2. 현재 50번의 조합 중 가장 성능이 좋은 모델 저장
#         model.set_params(**params_with_loss)
#         model.fit(X_train, y_train)
#         best_model_checkpoint_path = os.path.join(save_dir, f"best_model_{model_counter}.cbm")
#         model.save_model(best_model_checkpoint_path)
#         print(f"Best Model Checkpoint Saved: {best_model_checkpoint_path} for Current 50 Iterations")

# tqdm_iter.close()

# # 최종 결과 저장
# print(f"최적 파라미터: {best_params}, 최적 RMSE: {best_score}")
# final_model_path = os.path.join(save_dir, "catboost_best_model.cbm")
# model.set_params(**best_params)
# model.fit(X_train, y_train)
# model.save_model(final_model_path)
# print(f"최적 모델이 '{final_model_path}'에 저장되었습니다.")


In [ ]:
# import os
# from catboost import CatBoostRegressor
# import pandas as pd

# # 중복된 컬럼 제거
# train = train.loc[:, ~train.columns.duplicated()]

# # 범주형 및 수치형 변수 분리
# categorical_columns = train.select_dtypes(include=['object', 'category']).columns
# cat_features = [train.columns.get_loc(col) for col in categorical_columns]

# # 데이터 준비
# X_train = train.drop(columns=['utility_agent1'])  # 'utility_agent1'을 타겟으로 지정
# y_train = train['utility_agent1']

# # 저장 디렉토리 생성
# save_dir = 'model_checkpoints'
# os.makedirs(save_dir, exist_ok=True)

# # CatBoost 모델 초기화
# model = CatBoostRegressor(
#     loss_function='RMSE',
#     iterations=1000,
#     depth=10,
#     learning_rate=0.1,
#     cat_features=cat_features,
#     verbose=200,
#     task_type='GPU',
#     devices='0'
# )

# # 하이퍼파라미터 그리드 정의
# param_grid = {
#     'iterations': [500, 1000, 1500],
#     'depth': [6, 8, 10],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'l2_leaf_reg': [3, 5, 7],
# }

# # Grid Search 수행 및 각 반복마다 모델 저장
# grid_search_result = {}
# best_rmse = float('inf')
# best_params = None
# iteration = 0

# for params in model.grid_search(param_grid, X=X_train, y=y_train, cv=3, verbose=10, refit=False)['cv_results']:
#     iteration += 1
#     rmse = params['rmse_mean']
#     print(f"Iteration {iteration}: Params: {params}, RMSE: {rmse}")

#     # 모델 저장
#     model.set_params(**params)
#     model.fit(X_train, y_train)
#     save_path = os.path.join(save_dir, f'model_iteration_{iteration}.cbm')
#     model.save_model(save_path)
#     print(f"Model saved: {save_path} with RMSE: {rmse}")

#     # 최적 모델 갱신
#     if rmse < best_rmse:
#         best_rmse = rmse
#         best_params = params

# # 최적 모델 저장
# model.set_params(**best_params)
# model.fit(X_train, y_train)
# final_model_path = os.path.join(save_dir, 'best_model.cbm')
# model.save_model(final_model_path)
# print(f"Final best model saved: {final_model_path} with RMSE: {best_rmse}")


In [ ]:
# import os
# from catboost import CatBoostRegressor
# import pandas as pd

# # 중복된 컬럼 제거
# train = train.loc[:, ~train.columns.duplicated()]

# # 범주형 및 수치형 변수 분리
# categorical_columns = train.select_dtypes(include=['object', 'category']).columns
# cat_features = [train.columns.get_loc(col) for col in categorical_columns]

# # 데이터 준비
# X_train = train.drop(columns=['utility_agent1'])  # 'utility_agent1'을 타겟으로 지정
# y_train = train['utility_agent1']

# # 저장 디렉토리 생성
# save_dir = 'model_checkpoints'
# os.makedirs(save_dir, exist_ok=True)

# # 하이퍼파라미터 그리드 정의
# param_grid={
#     'iterations': [500, 1000, 1500],
#     'depth': [6, 8, 10],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'l2_leaf_reg': [3, 5, 7]
# }

# # 반복 실행
# iteration = 0
# best_rmse = float('inf')
# best_params = None

# # 모든 하이퍼파라미터 조합 생성
# from itertools import product
# param_combinations = list(product(
#     param_grid['iterations'],
#     param_grid['depth'],
#     param_grid['learning_rate'],
#     param_grid['l2_leaf_reg']
# ))

# for params in param_combinations:
#     iteration += 1
#     params_dict = {
#         'iterations': params[0],
#         'depth': params[1],
#         'learning_rate': params[2],
#         'l2_leaf_reg': params[3],
#         'loss_function': 'RMSE'
#     }

#     # 모델 초기화
#     model = CatBoostRegressor(
#         **params_dict,
#         cat_features=cat_features,
#         task_type='GPU',
#         devices='0',
#         verbose=200
#     )

#     # 모델 학습
#     model.fit(X_train, y_train)

#     # RMSE 계산
#     rmse = model.get_best_score()['learn']['RMSE']
#     print(f"Iteration {iteration}: Params: {params_dict}, RMSE: {rmse}")

#     # 모델 저장
#     save_path = os.path.join(save_dir, f'model_iteration_{iteration}.cbm')
#     model.save_model(save_path)
#     print(f"Model saved: {save_path} with RMSE: {rmse}")

#     # 최적 모델 업데이트
#     if rmse < best_rmse:
#         best_rmse = rmse
#         best_params = params_dict

# # 최적 모델 저장
# final_model = CatBoostRegressor(
#     **best_params,
#     cat_features=cat_features,
#     task_type='GPU',
#     devices='0',
#     verbose=200
# )
# final_model.fit(X_train, y_train)
# final_model_path = os.path.join(save_dir, 'best_model.cbm')
# final_model.save_model(final_model_path)
# print(f"Final best model saved: {final_model_path} with RMSE: {best_rmse}")
# /content/drive/MyDrive/MCTS/model_checkpoints/model_iteration_2.cbm


In [ ]:
# import os
# import random
# import numpy as np
# from catboost import Pool, CatBoostRegressor, cv
# import pandas as pd

# # 랜덤 시드 설정
# random_seed = 42
# random.seed(random_seed)
# np.random.seed(random_seed)

# # 중복된 컬럼 제거
# train = train.loc[:, ~train.columns.duplicated()]

# # 범주형 및 수치형 변수 분리
# categorical_columns = train.select_dtypes(include=['object', 'category']).columns
# cat_features = [train.columns.get_loc(col) for col in categorical_columns]

# # 데이터 준비
# X_train = train.drop(columns=['utility_agent1'])  # 'utility_agent1'을 타겟으로 지정
# y_train = train['utility_agent1']

# # CatBoost Pool 생성
# train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)

# # 저장 디렉토리 생성
# save_dir = '/content/drive/MyDrive/MCTS/model_checkpoints'
# os.makedirs(save_dir, exist_ok=True)

# # 하이퍼파라미터 공간 정의
# param_space = {
#     'iterations': [500, 1000, 1500],
#     'depth': [6, 8, 10, 12],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'l2_leaf_reg': [1, 4, 7, 10]
# }

# # 랜덤 샘플링을 위한 함수 정의
# def sample_params(param_space):
#     return {
#         'iterations': random.choice(param_space['iterations']),
#         'depth': random.choice(param_space['depth']),
#         'learning_rate': random.choice(param_space['learning_rate']),
#         'l2_leaf_reg': random.choice(param_space['l2_leaf_reg']),
#         'loss_function': 'RMSE',
#         'random_seed': random_seed  # 시드 고정
#     }

# # 랜덤 서치 설정
# n_search = 100  # 탐색 횟수
# best_rmse = float('inf')
# best_params = None

# # 랜덤 서치 시작
# for i in range(1, n_search + 1):
#     params_dict = sample_params(param_space)

#     # 교차검증 실행
#     cv_results = cv(
#         pool=train_pool,
#         params=params_dict,
#         fold_count=3,  # 3-Fold 교차검증
#         verbose=False,
#         early_stopping_rounds=50
#     )

#     # RMSE 계산 (test의 최적 평균 값)
#     mean_rmse = cv_results['test-RMSE-mean'].min()
#     print(f"Search {i}: Params: {params_dict}, Mean RMSE: {mean_rmse}")

#     # 최적 모델 업데이트
#     if mean_rmse < best_rmse:
#         best_rmse = mean_rmse
#         best_params = params_dict

#     # 10번마다 모델 저장
#     if i % 10 == 0:
#         # 모델 초기화 및 학습
#         model = CatBoostRegressor(
#             **params_dict,
#             cat_features=cat_features,
#             task_type='GPU',
#             devices='0',
#             verbose=200
#         )
#         model.fit(X_train, y_train)

#         # 모델 저장
#         save_path = os.path.join(save_dir, f'model_search_{i}.cbm')
#         model.save_model(save_path)
#         print(f"Model saved at search {i}: {save_path}")

# # 최적 하이퍼파라미터로 최종 모델 학습 및 저장
# final_model = CatBoostRegressor(**best_params, cat_features=cat_features, task_type='GPU', devices='0', verbose=200)
# final_model.fit(X_train, y_train)
# final_model_path = os.path.join(save_dir, 'best_model.cbm')
# final_model.save_model(final_model_path)
# print(f"Final best model saved: {final_model_path} with RMSE: {best_rmse}")


In [ ]:
# import os
# import random
# import numpy as np
# from catboost import Pool, CatBoostRegressor, cv
# import pandas as pd

# # 랜덤 시드 설정
# random_seed = 42
# random.seed(random_seed)
# np.random.seed(random_seed)

# # 중복된 컬럼 제거
# train = train.loc[:, ~train.columns.duplicated()]

# # 범주형 및 수치형 변수 분리
# categorical_columns = train.select_dtypes(include=['object', 'category']).columns
# cat_features = [train.columns.get_loc(col) for col in categorical_columns]

# # 데이터 준비
# X_train = train.drop(columns=['utility_agent1'])  # 'utility_agent1'을 타겟으로 지정
# y_train = train['utility_agent1']

# # CatBoost Pool 생성
# train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)

# # 저장 디렉토리 생성
# save_dir = '/content/drive/MyDrive/MCTS/model_checkpoints'
# os.makedirs(save_dir, exist_ok=True)

# # 하이퍼파라미터 공간 정의
# param_space = {
#     'iterations': [500, 1000, 1500],
#     'depth': [6, 8, 10, 12],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'l2_leaf_reg': [1, 4, 7, 10]
# }

# # 랜덤 샘플링을 위한 함수 정의
# def sample_params(param_space):
#     return {
#         'iterations': random.choice(param_space['iterations']),
#         'depth': random.choice(param_space['depth']),
#         'learning_rate': random.choice(param_space['learning_rate']),
#         'l2_leaf_reg': random.choice(param_space['l2_leaf_reg']),
#         'loss_function': 'RMSE',
#         'task_type': 'GPU',  # GPU 사용 강제
#         'devices': '0',      # 첫 번째 GPU 사용
#         'random_seed': random_seed
#     }

# # 랜덤 서치 설정
# n_search = 100  # 탐색 횟수
# best_rmse = float('inf')
# best_params = None

# # 랜덤 서치 시작
# for i in range(1, n_search + 1):
#     params_dict = sample_params(param_space)

#     # 교차검증 실행
#     cv_results = cv(
#         pool=train_pool,
#         params=params_dict,
#         fold_count=3,  # 3-Fold 교차검증
#         verbose=True,  # 학습 진행 로그 출력
#         early_stopping_rounds=50
#     )

#     # RMSE 계산 (test의 최적 평균 값)
#     mean_rmse = cv_results['test-RMSE-mean'].min()
#     print(f"Search {i}: Params: {params_dict}, Mean RMSE: {mean_rmse}")

#     # 최적 모델 업데이트
#     if mean_rmse < best_rmse:
#         best_rmse = mean_rmse
#         best_params = params_dict

#     # 10번마다 모델 저장
#     if i % 10 == 0:
#         # 모델 초기화 및 학습
#         model = CatBoostRegressor(
#             **params_dict,
#             cat_features=cat_features,
#             verbose=200
#         )
#         model.fit(X_train, y_train)

#         # 모델 저장
#         save_path = os.path.join(save_dir, f'model_search_{i}.cbm')
#         model.save_model(save_path)
#         print(f"Model saved at search {i}: {save_path}")

# # 최적 하이퍼파라미터로 최종 모델 학습 및 저장
# final_model = CatBoostRegressor(**best_params, cat_features=cat_features, verbose=200)
# final_model.fit(X_train, y_train)
# final_model_path = os.path.join(save_dir, 'best_model.cbm')
# final_model.save_model(final_model_path)
# print(f"Final best model saved: {final_model_path} with RMSE: {best_rmse}")


In [ ]:
import os
import random
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

# 랜덤 시드 설정
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# 중복된 컬럼 제거
train = train.loc[:, ~train.columns.duplicated()]

# 범주형 및 수치형 변수 분리
categorical_columns = train.select_dtypes(include=['object', 'category']).columns
cat_features = [train.columns.get_loc(col) for col in categorical_columns]

# 데이터 준비
X = train.drop(columns=['utility_agent1'])  # 'utility_agent1'을 타겟으로 지정
y = train['utility_agent1']

# 데이터셋 분리: Train/Test Split (80:20 비율)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# CatBoost Pool 생성
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool = Pool(data=X_val, label=y_val, cat_features=cat_features)

# 저장 디렉토리 생성
save_dir = '/content/drive/MyDrive/MCTS/model_checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 하이퍼파라미터 공간 정의
param_space = {
    'iterations': [500, 1000, 1500],
    'depth': [6, 8, 10, 12],
    'learning_rate': [0.05, 0.1, 0.2],
    'l2_leaf_reg': [1, 4, 7, 10]
}

# 랜덤 샘플링을 위한 함수 정의
def sample_params(param_space):
    return {
        'iterations': random.choice(param_space['iterations']),
        'depth': random.choice(param_space['depth']),
        'learning_rate': random.choice(param_space['learning_rate']),
        'l2_leaf_reg': random.choice(param_space['l2_leaf_reg']),
        'loss_function': 'RMSE',
        'task_type': 'GPU',  # GPU 사용 강제
        'devices': '0',      # 첫 번째 GPU 사용
        'random_seed': random_seed
    }

# 랜덤 서치 설정
n_search = 100  # 탐색 횟수
best_rmse = float('inf')
best_params = None

# 랜덤 서치 시작
for i in range(1, n_search + 1):
    params_dict = sample_params(param_space)

    # 모델 학습
    model = CatBoostRegressor(**params_dict, cat_features=cat_features, verbose=200)
    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50)

    # 검증 세트 RMSE 계산
    val_predictions = model.predict(X_val)
    rmse = np.sqrt(np.mean((val_predictions - y_val) ** 2))
    print(f"Search {i}: Params: {params_dict}, Validation RMSE: {rmse}")

    # 최적 모델 업데이트
    if rmse < best_rmse:
        best_rmse = rmse
        best_params = params_dict

    # 10번마다 모델 저장
    if i % 10 == 0:
        save_path = os.path.join(save_dir, f'model_search_{i}.cbm')
        model.save_model(save_path)
        print(f"Model saved at search {i}: {save_path}")

# 최적 하이퍼파라미터로 최종 모델 학습 및 저장
final_model = CatBoostRegressor(**best_params, cat_features=cat_features, verbose=200)
final_model.fit(train_pool, eval_set=val_pool)
final_model_path = os.path.join(save_dir, 'best_model.cbm')
final_model.save_model(final_model_path)
print(f"Final best model saved: {final_model_path} with RMSE: {best_rmse}")


0:	learn: 0.6144795	test: 0.6157611	best: 0.6157611 (0)	total: 21.4ms	remaining: 32.1s
200:	learn: 0.4407563	test: 0.4411416	best: 0.4411416 (200)	total: 2.78s	remaining: 18s
400:	learn: 0.4129897	test: 0.4140232	best: 0.4140232 (400)	total: 5.3s	remaining: 14.5s
600:	learn: 0.3974988	test: 0.3991946	best: 0.3991946 (600)	total: 7.84s	remaining: 11.7s
800:	learn: 0.3873517	test: 0.3898288	best: 0.3898288 (800)	total: 10.4s	remaining: 9.08s
1000:	learn: 0.3798472	test: 0.3830826	best: 0.3830826 (1000)	total: 13s	remaining: 6.49s
1200:	learn: 0.3739331	test: 0.3778985	best: 0.3778985 (1200)	total: 15.7s	remaining: 3.92s
1400:	learn: 0.3690796	test: 0.3737855	best: 0.3737855 (1400)	total: 18.4s	remaining: 1.3s
1499:	learn: 0.3669241	test: 0.3719994	best: 0.3719994 (1499)	total: 19.6s	remaining: 0us
bestTest = 0.3719994172
bestIteration = 1499
Search 1: Params: {'iterations': 1500, 'depth': 6, 'learning_rate': 0.05, 'l2_leaf_reg': 7, 'loss_function': 'RMSE', 'task_type': 'GPU', 'devices': 

In [ ]:
import os
import random
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

# 랜덤 시드 설정
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# 중복된 컬럼 제거
train = train.loc[:, ~train.columns.duplicated()]

# 범주형 및 수치형 변수 분리
categorical_columns = train.select_dtypes(include=['object', 'category']).columns
cat_features = [train.columns.get_loc(col) for col in categorical_columns]

# 데이터 준비
X = train.drop(columns=['utility_agent1'])  # 'utility_agent1'을 타겟으로 지정
y = train['utility_agent1']

# 데이터셋 분리: Train/Test Split (80:20 비율)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# CatBoost Pool 생성
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool = Pool(data=X_val, label=y_val, cat_features=cat_features)

# 저장 디렉토리 생성
save_dir = '/content/drive/MyDrive/MCTS/model_checkpoints_rs2'
os.makedirs(save_dir, exist_ok=True)

# 하이퍼파라미터 공간 정의
param_space = {
    'iterations': [500, 1000, 1500],
    'depth': [6, 8, 10, 12],
    'learning_rate': [0.05, 0.1, 0.2],
    'l2_leaf_reg': [1, 4, 7, 10],
    'border_count': [32, 64, 128]
}

# 랜덤 샘플링을 위한 함수 정의
def sample_params(param_space):
    return {
        'iterations': random.choice(param_space['iterations']),
        'depth': random.choice(param_space['depth']),
        'learning_rate': random.choice(param_space['learning_rate']),
        'l2_leaf_reg': random.choice(param_space['l2_leaf_reg']),
        'loss_function': 'RMSE',
        'task_type': 'GPU',  # GPU 사용 강제
        'devices': '0',      # 첫 번째 GPU 사용
        'random_seed': random_seed
    }

# 랜덤 서치 설정
n_search = 200  # 탐색 횟수
best_rmse = float('inf')
best_params = None

# 랜덤 서치 시작
for i in range(1, n_search + 1):
    params_dict = sample_params(param_space)

    # 모델 학습
    model = CatBoostRegressor(**params_dict, cat_features=cat_features, verbose=200)
    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50)

    # 검증 세트 RMSE 계산
    val_predictions = model.predict(X_val)
    rmse = np.sqrt(np.mean((val_predictions - y_val) ** 2))
    print(f"Search {i}: Params: {params_dict}, Validation RMSE: {rmse}")

    # 최적 모델 업데이트
    if rmse < best_rmse:
        best_rmse = rmse
        best_params = params_dict

    # 10번마다 모델 저장
    if i % 10 == 0:
        save_path = os.path.join(save_dir, f'model_search_{i}.cbm')
        model.save_model(save_path)
        print(f"Model saved at search {i}: {save_path}")

# 최적 하이퍼파라미터로 최종 모델 학습 및 저장
final_model = CatBoostRegressor(**best_params, cat_features=cat_features, verbose=200)
final_model.fit(train_pool, eval_set=val_pool)
final_model_path = os.path.join(save_dir, 'best_model.cbm')
final_model.save_model(final_model_path)
print(f"Final best model saved: {final_model_path} with RMSE: {best_rmse}")


0:	learn: 0.6144795	test: 0.6157611	best: 0.6157611 (0)	total: 15.2ms	remaining: 22.8s
200:	learn: 0.4407563	test: 0.4411416	best: 0.4411416 (200)	total: 2.62s	remaining: 16.9s
400:	learn: 0.4129897	test: 0.4140232	best: 0.4140232 (400)	total: 5.48s	remaining: 15s
600:	learn: 0.3974988	test: 0.3991946	best: 0.3991946 (600)	total: 8.2s	remaining: 12.3s
800:	learn: 0.3873517	test: 0.3898288	best: 0.3898288 (800)	total: 10.7s	remaining: 9.37s
1000:	learn: 0.3798472	test: 0.3830826	best: 0.3830826 (1000)	total: 13.3s	remaining: 6.61s
1200:	learn: 0.3739331	test: 0.3778985	best: 0.3778985 (1200)	total: 15.8s	remaining: 3.93s
1400:	learn: 0.3690796	test: 0.3737855	best: 0.3737855 (1400)	total: 18.4s	remaining: 1.3s
1499:	learn: 0.3669241	test: 0.3719994	best: 0.3719994 (1499)	total: 19.7s	remaining: 0us
bestTest = 0.3719994172
bestIteration = 1499
Search 1: Params: {'iterations': 1500, 'depth': 6, 'learning_rate': 0.05, 'l2_leaf_reg': 7, 'loss_function': 'RMSE', 'task_type': 'GPU', 'devices'